In [ ]:
import pandas as pd
import os
from onekey_algo.custom.components.ugly import drop_error
import numpy as np
from onekey_algo.custom.components import metrics
from onekey_algo.custom.components.comp1 import draw_roc
import matplotlib.pyplot as plt
from onekey_algo import get_param_in_cwd

os.makedirs('img', exist_ok=True)
metric_results = []
dtl = []

model_root = get_param_in_cwd('model_root_sup')
for model in os.listdir(model_root):
    train_log_path = os.path.join(model_root, rf'{model}/viz/BST_TRAIN_RESULTS.txt')
#     train_log_path = rf'D:\20230713-BiQiu\Pathology\models/{model}/train/Epoch-1.txt'
    train_log = pd.read_csv(train_log_path, names=['fname', 'pred_score', 'pred_label', 'gt'], sep='\t')
    train_log['group'] = 'train'
    val_log_path = os.path.join(model_root, rf'{model}/viz/BST_VAL_RESULTS.txt')
#     val_log_path = rf'D:\20230713-BiQiu\Pathology\models/{model}/valid/Epoch-1.txt'
    val_log = pd.read_csv(val_log_path, names=['fname', 'pred_score', 'pred_label', 'gt'], sep='\t')
    val_log['group'] = 'test'
    log = pd.concat([train_log, val_log], axis=0)
    
    all_pred = []
    all_gt = []
#     log = pd.merge(log, data, left_on='fname', right_on='fpath', how='inner')
    for subset in ['train', 'test']:
        val_log = log[log['group'] == subset]
        # ----
#         if subset in ['val', 'test1', 'test2']:
#             sel_idx, = drop_error([val_log['pred_label']], [val_log['gt']], [val_log['pred_score']], ratio=ra[subset])
#             print(model, subset, val_log.shape, val_log[sel_idx].shape)
#             val_log = val_log[sel_idx]
#         # ----
#         display(val_log)
        ul_labels = np.unique(val_log['pred_label'])
        pred_score = list(map(lambda x: x[0] if x[1] == 1 else 1-x[0], np.array(val_log[['pred_score', 'pred_label']])))
        gt = [1 if gt_ == 1 else 0 for gt_ in np.array(val_log['gt'])]
#         print(len(pred_score))
        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = metrics.analysis_pred_binary(gt, pred_score, use_youden=False)
        ci = f"{ci[0]:.4f}-{ci[1]:.4f}"
        metric_results.append([model, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, subset])
        
        all_pred.append(np.array(list(map(lambda x: (1-x[0], x[0]) if x[1] == 1 else (x[0], 1-x[0]), 
                                          np.array(val_log[['pred_score', 'pred_label']])))))
        all_gt.append(gt)
        
    draw_roc(all_gt, all_pred, labels=['Train', 'Test'], title=f"Model: {model}")
    plt.savefig(f'img/DLSup_model_{model}_roc.svg', bbox_inches = 'tight')
    plt.show()
metrics = pd.DataFrame(metric_results, 
             columns=['ModelName', 'Acc', 'AUC', '95% CI', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 
                      'Precision', 'Recall', 'F1', 'Threshold', 'Cohort'])
metrics